In [1]:
import torch
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F

In [2]:
training_data = torchvision.datasets.MNIST('/files/', train=True, download=True, transform=torchvision.transforms.ToTensor())
test_data = torchvision.datasets.MNIST('/files/', train=False, download=True, transform=torchvision.transforms.ToTensor())

In [3]:
n_epochs = 10
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.9
log_interval = 100

In [4]:
train_loader = torch.utils.data.DataLoader(training_data, batch_size=batch_size_train, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size_test, shuffle=True)

In [5]:
examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)
image_dimensions = example_data[0][0].size()

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, padding=2)
        self.sigmoid1 = nn.Sigmoid()
        self.avg1 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.sigmoid2 = nn.Sigmoid()
        self.avg2 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(16*5*5, 120)
        self.sigmoid3 = nn.Sigmoid()
        self.fc2 = nn.Linear(120, 84)
        self.sigmoid4 = nn.Sigmoid()
        self.fc3 = nn.Linear(84, 10)
    def forward(self, x):
        x = self.conv1(x)
        x = self.sigmoid1(x)
        x = self.avg1(x)
        x = self.conv2(x)
        x = self.sigmoid2(x)
        x = self.avg2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.sigmoid3(x)
        x = self.fc2(x)
        x = self.sigmoid4(x)
        x = self.fc3(x)
        return x


In [7]:
net = Net()
net.cuda()
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)

In [8]:
train_losses = []
train_counter = []
test_losses = []

In [9]:
def train(epoch):
    net.train()
    for batch_idx, (x, y) in enumerate(train_loader):
        x = x.cuda()
        y = y.cuda()
        optimizer.zero_grad()
        y_hat = net(x)
        l = loss(y_hat, y)
        l.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(x), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), l.item()))
            train_losses.append(l.item())
            train_counter.append(
                (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))

In [10]:
def test():
  net.eval()
  test_loss = 0
  correct = 0
  test_lenth = 0
  with torch.no_grad():
    for x, y in test_loader:
      x = x.cuda()
      y = y.cuda()
      y_hat = net(x)
      test_loss +=  loss(y_hat, y).item()
      test_lenth += 1
      pred = y_hat.data.max(1, keepdim=True)[1]
      correct += pred.eq(y.data.view_as(pred)).sum()
  test_loss /= test_lenth
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [11]:
for epoch in range(0, 20):
  train(epoch)
  test()

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.403898
Train Epoch: 0 [6400/60000 (11%)]	Loss: 2.308683
Train Epoch: 0 [12800/60000 (21%)]	Loss: 2.299219
Train Epoch: 0 [19200/60000 (32%)]	Loss: 2.279332
Train Epoch: 0 [25600/60000 (43%)]	Loss: 2.290752
Train Epoch: 0 [32000/60000 (53%)]	Loss: 2.324804
Train Epoch: 0 [38400/60000 (64%)]	Loss: 2.297505
Train Epoch: 0 [44800/60000 (75%)]	Loss: 2.299642
Train Epoch: 0 [51200/60000 (85%)]	Loss: 2.298535
Train Epoch: 0 [57600/60000 (96%)]	Loss: 2.280849

Test set: Avg. loss: 2.3036, Accuracy: 1135/10000 (11%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.297695
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.304899
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.289840
Train Epoch: 1 [19200/60000 (32%)]	Loss: 2.295710
Train Epoch: 1 [25600/60000 (43%)]	Loss: 2.280529
Train Epoch: 1 [32000/60000 (53%)]	Loss: 2.294420
Train Epoch: 1 [38400/60000 (64%)]	Loss: 2.313507
Train Epoch: 1 [44800/60000 (75%)]	Loss: 2.310415
Train Epoch: 1 [51200/60000 (85%)]	Loss: 2.307390
Trai

In [12]:
n_epochs = 10
batch_size_train = 128
batch_size_test = 1000
learning_rate = 0.9
log_interval = 100
train_loader = torch.utils.data.DataLoader(training_data, batch_size=batch_size_train, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size_test, shuffle=True)
net = Net()
net.cuda()
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)
for epoch in range(0, 20):
  train(epoch)
  test()

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.406199
Train Epoch: 0 [12800/60000 (21%)]	Loss: 2.310143
Train Epoch: 0 [25600/60000 (43%)]	Loss: 2.308496
Train Epoch: 0 [38400/60000 (64%)]	Loss: 2.292894
Train Epoch: 0 [51200/60000 (85%)]	Loss: 2.291936

Test set: Avg. loss: 2.3026, Accuracy: 1135/10000 (11%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.309949
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.303190
Train Epoch: 1 [25600/60000 (43%)]	Loss: 2.318685
Train Epoch: 1 [38400/60000 (64%)]	Loss: 2.307415
Train Epoch: 1 [51200/60000 (85%)]	Loss: 2.294907

Test set: Avg. loss: 2.3032, Accuracy: 1010/10000 (10%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 2.304772
Train Epoch: 2 [12800/60000 (21%)]	Loss: 2.299701
Train Epoch: 2 [25600/60000 (43%)]	Loss: 2.311692
Train Epoch: 2 [38400/60000 (64%)]	Loss: 2.298206
Train Epoch: 2 [51200/60000 (85%)]	Loss: 2.301968

Test set: Avg. loss: 2.3024, Accuracy: 982/10000 (10%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 2.294412
Train Epoch: 3 [12800/60000 (21%)]	Loss: 2.2837

In [13]:
n_epochs = 20
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.1
log_interval = 100
train_loader = torch.utils.data.DataLoader(training_data, batch_size=batch_size_train, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size_test, shuffle=True)
net = Net()
net.cuda()
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)
for epoch in range(0, n_epochs):
  train(epoch)
  test()

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.348299
Train Epoch: 0 [6400/60000 (11%)]	Loss: 2.320311
Train Epoch: 0 [12800/60000 (21%)]	Loss: 2.292415
Train Epoch: 0 [19200/60000 (32%)]	Loss: 2.309829
Train Epoch: 0 [25600/60000 (43%)]	Loss: 2.289732
Train Epoch: 0 [32000/60000 (53%)]	Loss: 2.313495
Train Epoch: 0 [38400/60000 (64%)]	Loss: 2.325651
Train Epoch: 0 [44800/60000 (75%)]	Loss: 2.301109
Train Epoch: 0 [51200/60000 (85%)]	Loss: 2.331421
Train Epoch: 0 [57600/60000 (96%)]	Loss: 2.321550

Test set: Avg. loss: 2.3097, Accuracy: 1009/10000 (10%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.293849
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.310735
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.316483
Train Epoch: 1 [19200/60000 (32%)]	Loss: 2.315705
Train Epoch: 1 [25600/60000 (43%)]	Loss: 2.303587
Train Epoch: 1 [32000/60000 (53%)]	Loss: 2.303764
Train Epoch: 1 [38400/60000 (64%)]	Loss: 2.305026
Train Epoch: 1 [44800/60000 (75%)]	Loss: 2.299360
Train Epoch: 1 [51200/60000 (85%)]	Loss: 2.283940
Trai

In [14]:
n_epochs = 20
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.5
log_interval = 100
train_loader = torch.utils.data.DataLoader(training_data, batch_size=batch_size_train, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size_test, shuffle=True)
net = Net()
net.cuda()
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)
for epoch in range(0, n_epochs):
  train(epoch)
  test()

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.316472
Train Epoch: 0 [6400/60000 (11%)]	Loss: 2.270283
Train Epoch: 0 [12800/60000 (21%)]	Loss: 2.293371
Train Epoch: 0 [19200/60000 (32%)]	Loss: 2.342576
Train Epoch: 0 [25600/60000 (43%)]	Loss: 2.287371
Train Epoch: 0 [32000/60000 (53%)]	Loss: 2.309337
Train Epoch: 0 [38400/60000 (64%)]	Loss: 2.293620
Train Epoch: 0 [44800/60000 (75%)]	Loss: 2.309884
Train Epoch: 0 [51200/60000 (85%)]	Loss: 2.302358
Train Epoch: 0 [57600/60000 (96%)]	Loss: 2.304600

Test set: Avg. loss: 2.3044, Accuracy: 1135/10000 (11%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.313305
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.281745
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.305881
Train Epoch: 1 [19200/60000 (32%)]	Loss: 2.317631
Train Epoch: 1 [25600/60000 (43%)]	Loss: 2.306608
Train Epoch: 1 [32000/60000 (53%)]	Loss: 2.313788
Train Epoch: 1 [38400/60000 (64%)]	Loss: 2.301371
Train Epoch: 1 [44800/60000 (75%)]	Loss: 2.309585
Train Epoch: 1 [51200/60000 (85%)]	Loss: 2.301771
Trai

In [15]:
n_epochs = 20
batch_size_train = 64
batch_size_test = 1000
learning_rate = 5
log_interval = 100
train_loader = torch.utils.data.DataLoader(training_data, batch_size=batch_size_train, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size_test, shuffle=True)
net = Net()
net.cuda()
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)
for epoch in range(0, n_epochs):
  train(epoch)
  test()

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.401287
Train Epoch: 0 [6400/60000 (11%)]	Loss: 2.303521
Train Epoch: 0 [12800/60000 (21%)]	Loss: 2.306221
Train Epoch: 0 [19200/60000 (32%)]	Loss: 2.350470
Train Epoch: 0 [25600/60000 (43%)]	Loss: 2.292216
Train Epoch: 0 [32000/60000 (53%)]	Loss: 2.317939
Train Epoch: 0 [38400/60000 (64%)]	Loss: 2.378468
Train Epoch: 0 [44800/60000 (75%)]	Loss: 2.297895
Train Epoch: 0 [51200/60000 (85%)]	Loss: 2.355414
Train Epoch: 0 [57600/60000 (96%)]	Loss: 2.323107

Test set: Avg. loss: 2.3481, Accuracy: 1135/10000 (11%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.351783
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.305143
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.314730
Train Epoch: 1 [19200/60000 (32%)]	Loss: 2.320455
Train Epoch: 1 [25600/60000 (43%)]	Loss: 2.373002
Train Epoch: 1 [32000/60000 (53%)]	Loss: 2.375511
Train Epoch: 1 [38400/60000 (64%)]	Loss: 2.356993
Train Epoch: 1 [44800/60000 (75%)]	Loss: 2.314435
Train Epoch: 1 [51200/60000 (85%)]	Loss: 2.316882
Trai